# Spark 101 Exercise

In [1]:

import pyspark
#create the spark session
spark = pyspark.sql.SparkSession.builder.getOrCreate()
#datafrane stuff
import pandas as pd
import numpy as np
#import data sets
from pydataset import data
from vega_datasets import data

#import functions
from pyspark.sql.functions import col,min, max, sum, count, mean, avg,round,expr,concat,lit
from pyspark.sql.functions import regexp_extract, regexp_replace,when,month, year, quarter




Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/08/22 15:04:30 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# 1. Create a spark data frame that contains your favorite programming languages.

In [2]:
lang_list=('R','Python','JavaScrip','Java','Ruby','Rust','SQL','HTML','CSS')
language_df = pd.DataFrame(lang_list,columns=['language']
    
)

language_df.head()

,language
0,R
1,Python
2,JavaScrip
3,Java
4,Ruby


In [3]:
df = spark.createDataFrame(language_df)

In [4]:
df.show()

+---------+
| language|
+---------+
|        R|
|   Python|
|JavaScrip|
|     Java|
|     Ruby|
|     Rust|
|      SQL|
|     HTML|
|      CSS|
+---------+



In [5]:
df.printSchema()

root
 |-- language: string (nullable = true)



In [6]:
df.count(),len(df.columns)

(9, 1)

In [7]:
df.show(5)

+---------+
| language|
+---------+
|        R|
|   Python|
|JavaScrip|
|     Java|
|     Ruby|
+---------+
only showing top 5 rows



# 2. Load the mpg dataset as a spark dataframe.

In [8]:
from pydataset import data
df = spark.createDataFrame(data('mpg'))
df

DataFrame[manufacturer: string, model: string, displ: double, year: bigint, cyl: bigint, trans: string, drv: string, cty: bigint, hwy: bigint, fl: string, class: string]

In [9]:
df.show(5)

+------------+-----+-----+----+---+----------+---+---+---+---+-------+
|manufacturer|model|displ|year|cyl|     trans|drv|cty|hwy| fl|  class|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+
|        audi|   a4|  1.8|1999|  4|  auto(l5)|  f| 18| 29|  p|compact|
|        audi|   a4|  1.8|1999|  4|manual(m5)|  f| 21| 29|  p|compact|
|        audi|   a4|  2.0|2008|  4|manual(m6)|  f| 20| 31|  p|compact|
|        audi|   a4|  2.0|2008|  4|  auto(av)|  f| 21| 30|  p|compact|
|        audi|   a4|  2.8|1999|  6|  auto(l5)|  f| 16| 26|  p|compact|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+
only showing top 5 rows



### a.Create 1 column of output that contains a message like the one below:
The 1999 audi a4 has a 4 cylinder engine.

In [10]:
year_manufacturer_model = df.select('year', 'manufacturer', 'model').collect()[0]
cylinder = df.select('cyl').collect()[0][0]

print(f"The {year_manufacturer_model['year']} {year_manufacturer_model['manufacturer']} {year_manufacturer_model['model']} has a {cylinder} cylinder engine.")


The 1999 audi a4 has a 4 cylinder engine.


In [11]:
df = df.withColumn(
    'engine_description',
    concat(
        lit("The "),
        df['year'],
        lit(" "),
        df['manufacturer'],
        lit(" "),
        df['model'],
        lit(" has a "),
        df['cyl'],
        lit(" cylinder engine.")
    )
)

df.show(truncate=False)

+------------+------------------+-----+----+---+----------+---+---+---+---+-------+--------------------------------------------------------------+
|manufacturer|model             |displ|year|cyl|trans     |drv|cty|hwy|fl |class  |engine_description                                            |
+------------+------------------+-----+----+---+----------+---+---+---+---+-------+--------------------------------------------------------------+
|audi        |a4                |1.8  |1999|4  |auto(l5)  |f  |18 |29 |p  |compact|The 1999 audi a4 has a 4 cylinder engine.                     |
|audi        |a4                |1.8  |1999|4  |manual(m5)|f  |21 |29 |p  |compact|The 1999 audi a4 has a 4 cylinder engine.                     |
|audi        |a4                |2.0  |2008|4  |manual(m6)|f  |20 |31 |p  |compact|The 2008 audi a4 has a 4 cylinder engine.                     |
|audi        |a4                |2.0  |2008|4  |auto(av)  |f  |21 |30 |p  |compact|The 2008 audi a4 has a 4 cylinder e

### For each vehicle.

b. Transform the trans column so that it only contains either manual or auto.

In [12]:
df.select(
    regexp_extract(df['trans'], r'^(manual|auto)', 1).alias('clean_trans')
).show(truncate=False)

+-----------+
|clean_trans|
+-----------+
|auto       |
|manual     |
|manual     |
|auto       |
|auto       |
|manual     |
|auto       |
|manual     |
|auto       |
|manual     |
|auto       |
|auto       |
|manual     |
|auto       |
|manual     |
|auto       |
|auto       |
|auto       |
|auto       |
|auto       |
+-----------+
only showing top 20 rows



In [13]:
df = df.withColumn(
    'engine_description',
    concat(
        lit("The "),
        df['year'],
        lit(" "),
        df['manufacturer'],
        lit(" "),
        df['model'],
        lit(" has a "),
        df['cyl'],
        lit(" cylinder engine.") )
)

df.show(truncate=False)

+------------+------------------+-----+----+---+----------+---+---+---+---+-------+--------------------------------------------------------------+
|manufacturer|model             |displ|year|cyl|trans     |drv|cty|hwy|fl |class  |engine_description                                            |
+------------+------------------+-----+----+---+----------+---+---+---+---+-------+--------------------------------------------------------------+
|audi        |a4                |1.8  |1999|4  |auto(l5)  |f  |18 |29 |p  |compact|The 1999 audi a4 has a 4 cylinder engine.                     |
|audi        |a4                |1.8  |1999|4  |manual(m5)|f  |21 |29 |p  |compact|The 1999 audi a4 has a 4 cylinder engine.                     |
|audi        |a4                |2.0  |2008|4  |manual(m6)|f  |20 |31 |p  |compact|The 2008 audi a4 has a 4 cylinder engine.                     |
|audi        |a4                |2.0  |2008|4  |auto(av)  |f  |21 |30 |p  |compact|The 2008 audi a4 has a 4 cylinder e

In [14]:
df = df.withColumn('clean_trans', regexp_extract(df['trans'], r'^(manual|auto)', 1))

df.show(truncate=False)

+------------+------------------+-----+----+---+----------+---+---+---+---+-------+--------------------------------------------------------------+-----------+
|manufacturer|model             |displ|year|cyl|trans     |drv|cty|hwy|fl |class  |engine_description                                            |clean_trans|
+------------+------------------+-----+----+---+----------+---+---+---+---+-------+--------------------------------------------------------------+-----------+
|audi        |a4                |1.8  |1999|4  |auto(l5)  |f  |18 |29 |p  |compact|The 1999 audi a4 has a 4 cylinder engine.                     |auto       |
|audi        |a4                |1.8  |1999|4  |manual(m5)|f  |21 |29 |p  |compact|The 1999 audi a4 has a 4 cylinder engine.                     |manual     |
|audi        |a4                |2.0  |2008|4  |manual(m6)|f  |20 |31 |p  |compact|The 2008 audi a4 has a 4 cylinder engine.                     |manual     |
|audi        |a4                |2.0  |2008|4 

## 3. Load the tips dataset as a spark dataframe.

In [15]:
df = spark.createDataFrame(data('tips'))
df

DataFrame[total_bill: double, tip: double, sex: string, smoker: string, day: string, time: string, size: bigint]

In [16]:
df.show(5)

+----------+----+------+------+---+------+----+
|total_bill| tip|   sex|smoker|day|  time|size|
+----------+----+------+------+---+------+----+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|
+----------+----+------+------+---+------+----+
only showing top 5 rows



### a. What percentage of observations are smokers?

In [17]:
df.groupBy().agg((count(when(df.smoker == 'Yes', True)) / count('*') * 100)).show()

+------------------------------------------------------------------+
|((count(CASE WHEN (smoker = Yes) THEN true END) / count(1)) * 100)|
+------------------------------------------------------------------+
|                                                38.114754098360656|
+------------------------------------------------------------------+



### b. Create a column that contains the tip percentage


In [18]:
df.groupBy().agg((count(df.tip) / count('total_bill'))).show()

+--------------------------------+
|(count(tip) / count(total_bill))|
+--------------------------------+
|                             1.0|
+--------------------------------+



In [19]:
total_tip = df.select(sum('tip')).first()[0]
df = df.withColumn('tip_percentage', (col('tip') / total_tip) * 100)

df.show()

+----------+----+------+------+---+------+----+-------------------+
|total_bill| tip|   sex|smoker|day|  time|size|     tip_percentage|
+----------+----+------+------+---+------+----+-------------------+
|     16.99|1.01|Female|    No|Sun|Dinner|   2| 0.1380573553131578|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|0.22690614833647715|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|0.47841657781787356|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|0.45244539216490326|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|0.49345252740643525|
|     25.29|4.71|  Male|    No|Sun|Dinner|   4| 0.6438120232920527|
|      8.77| 2.0|  Male|    No|Sun|Dinner|   2|0.27338090161021344|
|     26.88|3.12|  Male|    No|Sun|Dinner|   4|  0.426474206511933|
|     15.04|1.96|  Male|    No|Sun|Dinner|   2|0.26791328357800914|
|     14.78|3.23|  Male|    No|Sun|Dinner|   2|0.44151015610049477|
|     10.27|1.71|  Male|    No|Sun|Dinner|   2| 0.2337406708767325|
|     35.26| 5.0|Female|    No|Sun|Dinner|   4| 

### c. Calculate the average tip percentage for each combination of sex and smoker.

In [20]:
df.groupBy('sex', 'smoker').agg(avg('tip_percentage').alias('avg_tip_percentage')).show()

+------+------+-------------------+
|   sex|smoker| avg_tip_percentage|
+------+------+-------------------+
|  Male|    No|0.42557233137260037|
|Female|    No|0.37911349661260807|
|  Male|   Yes| 0.4170653471481815|
|Female|   Yes| 0.4007101276026068|
+------+------+-------------------+



4. Use the seattle weather dataset referenced in the lesson to answer the questions below.
- Convert the temperatures to fahrenheit.
- Which month has the most rain, on average?
- Which year was the windiest?
- What is the most frequent type of weather in January?
- What is the average high and low temperature on sunny days in July in 2013 and 2014?
- What percentage of days were rainy in q3 of 2015?
- For each year, find what percentage of days it rained (had non-zero precipitation).

In [21]:
from vega_datasets import data
weather = data.seattle_weather().assign(date=lambda df: df.date.astype(str))
df = spark.createDataFrame(weather)
df.show(6)

+----------+-------------+--------+--------+----+-------+
|      date|precipitation|temp_max|temp_min|wind|weather|
+----------+-------------+--------+--------+----+-------+
|2012-01-01|          0.0|    12.8|     5.0| 4.7|drizzle|
|2012-01-02|         10.9|    10.6|     2.8| 4.5|   rain|
|2012-01-03|          0.8|    11.7|     7.2| 2.3|   rain|
|2012-01-04|         20.3|    12.2|     5.6| 4.7|   rain|
|2012-01-05|          1.3|     8.9|     2.8| 6.1|   rain|
|2012-01-06|          2.5|     4.4|     2.2| 2.2|   rain|
+----------+-------------+--------+--------+----+-------+
only showing top 6 rows



In [22]:
df = df.withColumn('temp_max', (col('temp_max') * 9/5) + 32)
df = df.withColumn('temp_min', (col('temp_min') * 9/5) + 32)

df.show(5)

+----------+-------------+--------+--------+----+-------+
|      date|precipitation|temp_max|temp_min|wind|weather|
+----------+-------------+--------+--------+----+-------+
|2012-01-01|          0.0|   55.04|    41.0| 4.7|drizzle|
|2012-01-02|         10.9|   51.08|   37.04| 4.5|   rain|
|2012-01-03|          0.8|   53.06|   44.96| 2.3|   rain|
|2012-01-04|         20.3|   53.96|   42.08| 4.7|   rain|
|2012-01-05|          1.3|   48.02|   37.04| 6.1|   rain|
+----------+-------------+--------+--------+----+-------+
only showing top 5 rows



In [23]:
rainy_df=df.withColumn('month', month(df['date'])).groupBy('month').agg(avg('precipitation').alias('avg_rainfall')).orderBy('avg_rainfall', ascending=False)

In [24]:
rdf=rainy_df.select('month').first()[0]
#NOVEMBER
print("The raniest month is:", rdf)

The raniest month is: 11


In [25]:
wind_df=df.withColumn('year', year(df['date'])).groupBy('year').agg(avg('wind').alias('avg_wind')).orderBy('avg_wind', ascending=False)

In [26]:
year_df=wind_df.select('year').first()[0]
print("The windiest year is:", year_df)

The windiest year is: 2012


In [27]:
january_weather = df.filter(month(df['date']) == 1)
most_frequent_weather = january_weather.groupBy('weather').count().orderBy(col('count').desc()).first()[0]

print("The most frequent type of weather in January is:", most_frequent_weather)

The most frequent type of weather in January is: fog


In [28]:
from pyspark.sql.functions import month, year, avg
from pyspark.sql.types import IntegerType

In [29]:
df_filtered = df.filter((month(df['date']) == 7) & (year(df['date']).isin([2013, 2014])) & (col('weather') == 'sun'))

df_avg_temp = df_filtered.select(avg(col('temp_max')).cast(IntegerType()).alias('avg_high_temp'), avg(col('temp_min')).cast(IntegerType()).alias('avg_low_temp'))

avg_high_temp = df_avg_temp.select('avg_high_temp').first()[0]
avg_low_temp = df_avg_temp.select('avg_low_temp').first()[0]

print("Average high temperature on sunny days in July (2013 and 2014):", avg_high_temp)
print("Average low temperature on sunny days in July (2013 and 2014):", avg_low_temp)

Average high temperature on sunny days in July (2013 and 2014): 80
Average low temperature on sunny days in July (2013 and 2014): 57


In [30]:
q3_2015_rainy_days = df.filter((year(df['date']) == 2015) & (quarter(df['date']) == 3) & (col('weather') == 'rain'))
total_days_q3_2015 = df.filter((year(df['date']) == 2015) & (quarter(df['date']) == 3)).count()

rainy_days_percentage = (q3_2015_rainy_days.count() / total_days_q3_2015) * 100

print("Percentage of rainy days in Q3 2015:", rainy_days_percentage)

Percentage of rainy days in Q3 2015: 2.1739130434782608


In [31]:
yearly_rain_days = df.filter(col('precipitation') > 0).groupBy(year('date')).agg((count('*') / count('date') * 100).alias('rainy_days_percentage')).orderBy(year('date'))

yearly_rain_days.show()

+----------+---------------------+
|year(date)|rainy_days_percentage|
+----------+---------------------+
|      2012|                100.0|
|      2013|                100.0|
|      2014|                100.0|
|      2015|                100.0|
+----------+---------------------+

